In [2]:
from instacart_feature_transformation_script import FeatureGenerator,generate_test_set_features

In [4]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession,Window
import pyspark.sql.functions as F


In [5]:
spark = SparkSession.builder.appName("instamart_analysis") \
    .config("spark.driver.memory","25g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/03 05:54:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
orders_df = spark.read.csv('/kaggle/input/instacart-market-basket-analysis/orders.csv',header=True)
prior_product_orders = spark.read.csv("/kaggle/input/instacart-market-basket-analysis/order_products__prior.csv",header=True)
products_df = spark.read.csv("/kaggle/input/instacart-market-basket-analysis/products.csv",header=True)
train_product_orders= spark.read.csv("/kaggle/input/instacart-market-basket-analysis/order_products__train.csv",header=True)

In [7]:
prior_product_orders = prior_product_orders.select(
                                                       [F.col(col).cast("float").alias(col) for col in  prior_product_orders.columns] 
                                                        )
# convert the string col into the float for float.
orders_df = orders_df.select(
    [F.col(col).cast("float").alias(col) if col !='eval_set' else F.col(col).alias(col) for col in orders_df.columns]
)

train_product_orders = train_product_orders.select(
                                                       [F.col(col).cast("float").alias(col) for col in  prior_product_orders.columns] 
                                                        )

In [8]:
final_train_product_orders = train_product_orders.union(prior_product_orders)

In [9]:
final_train_orders_df = orders_df.filter(F.col("eval_set") != 'test').drop('eval_set')
test_orders_df = orders_df.filter(F.col("eval_set") == 'test').select("order_id","user_id")


### Checking test_orders_df

In [10]:
test_orders_df.count()

75000

In [11]:
# check how many distinct user,order pairs is there
test_orders_df.select("user_id").distinct().count()

75000

In [12]:
fet_gen = FeatureGenerator(final_train_product_orders,final_train_orders_df,products_df)

In [13]:
result_df = fet_gen.generate_user_related_features()
result_prod_df = fet_gen.generate_product_related_features()
result_user_prod_df = fet_gen.generate_user_product_related_features()
result_time_df=  fet_gen.generate_time_related_features()

In [14]:
final_prior_train_set = fet_gen.generate_all_types_of_features()

## checking training dataframe

In [15]:
# final_prior_train_set.count()

In [16]:
# final_prior_train_set.dropDuplicates().count()

In [20]:
# let's make test set from train set
test_set = (
    test_orders_df.select("user_id")
                    .join(
                        final_train_orders_df,on='user_id',how='left'
                    ).select("user_id","order_id")
                    .join(
                        prior_product_orders.select("order_id","product_id") , on ='order_id',how='left'
                    )
                    .select("user_id","product_id").distinct()
)

## checking test dataframe

In [21]:
# test_set.count()

4833292

In [22]:
# test_set.distinct().count()

4833292

In [23]:
featured_test_set = generate_test_set_features(result_df,result_prod_df,result_user_prod_df,result_time_df,test_set)

In [ ]:
final_prior_train_set.coalesce(1).write.csv("final_prior_train_set.csv")


In [ ]:
featured_test_set.coalesce(1).write.csv("featured_test_set.csv")


In [24]:
with open("train_set_columns.txt",'w') as f:
    for column in final_prior_train_set.columns:
        f.write('%s,'%column)


In [25]:

with open("test_set_columns.txt",'w') as f:
    for column in featured_test_set.columns:
        f.write('%s,'%column)